# load model from drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q transformers peft accelerate bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
lora_path = "/content/drive/MyDrive/FinBuddy/finbuddy_lora_final"  # Direct path

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, lora_path)
tokenizer = AutoTokenizer.from_pretrained(lora_path)

print("✅ Model loaded directly from Google Drive!")

✅ Model loaded directly from Google Drive!


## FINBUDDY CHATBOT

In [ ]:
!pip install -q gradio pandas

import pandas as pd
import torch
import gradio as gr
print("✅ Starting FinBuddy Chatbot Setup")

✅ Starting FinBuddy Chatbot Setup



# STEP 1: TRANSACTION ANALYSIS FUNCTIONS


In [ ]:
def analyze_transactions(df):
    """Analyze uploaded transaction CSV and return insights"""

    required_cols = ['date', 'amount', 'category', 'merchant']
    missing_cols = [col for col in required_cols if col not in df.columns]

    if missing_cols:
        return f"Error: Missing columns: {', '.join(missing_cols)}", None

    try:
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date')

        total_income = df[df['amount'] > 0]['amount'].sum()
        total_expenses = abs(df[df['amount'] < 0]['amount'].sum())

        days = (df['date'].max() - df['date'].min()).days
        months = max(days / 30, 1)

        monthly_income = total_income / months
        monthly_expenses = total_expenses / months
        net_monthly = monthly_income - monthly_expenses

        expense_by_cat = df[df['amount'] < 0].groupby('category')['amount'].sum().abs()
        top_categories = expense_by_cat.nlargest(5)

        current_balance = df.iloc[-1].get('balance', 'N/A')

        summary = f"""📊 **Financial Overview**

**Income & Expenses:**
- Average Monthly Income: ${monthly_income:,.2f}
- Average Monthly Expenses: ${monthly_expenses:,.2f}
- Net Monthly Cash Flow: ${net_monthly:,.2f}
- Current Balance: ${current_balance if current_balance != 'N/A' else 'Not provided'}

**Top Spending Categories:**
"""

        for cat, amount in top_categories.items():
            pct = (amount / total_expenses) * 100 if total_expenses > 0 else 0
            summary += f"- {cat}: ${amount:,.2f} ({pct:.1f}%)\n"

        summary += f"\n**Transaction Period:** {days} days ({months:.1f} months)\n"
        summary += f"**Total Transactions:** {len(df)}\n"

        return summary, df

    except Exception as e:
        return f"Error analyzing transactions: {str(e)}", None

def format_transactions_for_model(df, max_rows=15):
    """Format transactions for model input"""

    if df is None or len(df) == 0:
        return "No transactions available."

    recent = df.tail(max_rows)

    trans_text = "Recent Transactions:\n"
    for _, row in recent.iterrows():
        date_str = row['date'].strftime('%Y-%m-%d') if hasattr(row['date'], 'strftime') else str(row['date'])
        trans_text += f"- {date_str}: {row['category']} at {row['merchant']} (${row['amount']:,.2f})\n"

    return trans_text

# STEP 2: CHATBOT FUNCTIONS

In [ ]:
def chat(user_message, history):
    """Handle chat interactions with structured, complete responses"""
    global current_transactions

    if current_transactions is None:
        return history + [[user_message, "⚠️ Please upload your transaction CSV first!"]]

    if not user_message or user_message.strip() == "":
        return history

    try:
        trans_summary = format_transactions_for_model(current_transactions)

        # IMPROVED: More structured system prompt
        prompt = f"""<|im_start|>system
You are FinBuddy, a financial advisor. Always respond in this format:

**Situation:** [1 sentence analyzing their finances]

**Recommendation:**
- [Action 1 - be specific with numbers]
- [Action 2 - be specific with numbers]
- [Action 3 - be specific with numbers]

Keep each bullet under 20 words. Total response under 100 words.<|im_end|>
<|im_start|>user
{trans_summary}

User Question: {user_message}<|im_end|>
<|im_start|>assistant
"""

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

        model.eval()
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=250,              # INCREASED from 150
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3           # ADDED - prevents repetition
            )

        full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)

        try:
            assistant_response = full_response.split("<|im_start|>assistant")[-1].split("<|im_end|>")[0].strip()
        except:
            assistant_response = full_response.split("assistant")[-1].strip()

        # ADDED: Smart truncation at sentence boundary
        if len(assistant_response) > 800:  # ~130 words
            # Find last complete sentence
            for delimiter in ['. ', '.\n', '! ', '? ']:
                last_pos = assistant_response[:800].rfind(delimiter)
                if last_pos > 200:  # Keep meaningful content
                    assistant_response = assistant_response[:last_pos + 1]
                    break

        # ADDED: Ensure we have complete bullets
        lines = assistant_response.split('\n')
        complete_lines = []
        for line in lines:
            if line.strip().startswith('•') or line.strip().startswith('-'):
                # Check if bullet point is complete (ends with period or has >10 words)
                if line.strip().endswith(('.', '!', '?')) or len(line.split()) > 10:
                    complete_lines.append(line)
                else:
                    # Incomplete bullet - stop here
                    break
            else:
                complete_lines.append(line)

        assistant_response = '\n'.join(complete_lines).strip()

        if not assistant_response or len(assistant_response) < 10:
            assistant_response = "I apologize, I had trouble generating a response. Please try rephrasing your question."

        return history + [[user_message, assistant_response]]

    except Exception as e:
        return history + [[user_message, f"Error: {str(e)}"]]

# STEP 3: GRADIO INTERFACE

In [ ]:
print("\n" + "="*60)
print("LAUNCHING GRADIO INTERFACE")
print("="*60)

with gr.Blocks(title="FinBuddy") as demo:

    gr.Markdown("""
    # 💰 FinBuddy - Your Personal Finance Advisor

    Upload your transaction CSV and get personalized financial advice!

    **CSV Format:** `date, amount, category, merchant` (optional: `description`, `balance`)
    """)

    with gr.Row():
        with gr.Column(scale=1):
            file_upload = gr.File(label="📁 Upload CSV", file_types=[".csv"])
            transaction_summary = gr.Textbox(label="📊 Analysis", lines=15, interactive=False)
            upload_status = gr.Textbox(label="Status", interactive=False)
            reset_btn = gr.Button("🔄 Reset", variant="secondary")

        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                label="💬 Chat with FinBuddy",
                height=500
                # Removed type="messages" - causes compatibility issue
            )
            msg = gr.Textbox(
                label="Your Question",
                placeholder="Ask about budgeting, debt, savings...",
                lines=2
            )

            with gr.Row():
                send_btn = gr.Button("Send", variant="primary")
                clear_btn = gr.Button("Clear Chat")

    gr.Markdown("""
    ### Try asking:
    - "Give me a budgeting plan to start saving"
    - "Should I pay off debt or invest?"
    - "What's my biggest expense?"
    """)

    # Event handlers
    file_upload.change(
        fn=process_csv,
        inputs=[file_upload],
        outputs=[transaction_summary, upload_status]
    )

    msg.submit(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        None,
        [msg]
    )

    send_btn.click(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        None,
        [msg]
    )

    clear_btn.click(
        lambda: [],
        None,
        [chatbot]
    )

    reset_btn.click(
        fn=reset_chat,
        outputs=[chatbot, msg, upload_status]
    )

# Launch
demo.launch(share=True, debug=True)

print("\n✅ FinBuddy chatbot is live!")
print("📱 Upload your CSV and start chatting!")


LAUNCHING GRADIO INTERFACE


/tmp/ipython-input-2930572217.py:23: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2930572217.py:23: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bdccbc004ca3cbde39.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://bdccbc004ca3cbde39.gradio.live

✅ FinBuddy chatbot is live!
📱 Upload your CSV and start chatting!


# PERPLEXITY EVALUATION

In [ ]:
import os
import shutil
from datasets import Dataset
import json
from tqdm import tqdm
import torch

print("Starting evaluation setup...")

# STEP 1: Load eval data

if not os.path.exists('eval_data.jsonl'):
    print("Looking for eval data in Google Drive...")
    if os.path.exists('/content/drive/MyDrive/FinBuddy/eval_data.jsonl'):
        shutil.copy('/content/drive/MyDrive/FinBuddy/eval_data.jsonl', './')
        print("✅ Copied eval_data.jsonl from Drive")
    elif os.path.exists('/content/drive/MyDrive/FinBuddy/training_data.jsonl'):
        print("Using training_data.jsonl as fallback...")
        shutil.copy('/content/drive/MyDrive/FinBuddy/training_data.jsonl', './eval_data.jsonl')
    else:
        print("❌ No evaluation data found!")
        raise FileNotFoundError("Need eval_data.jsonl or training_data.jsonl in Drive")

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

eval_dataset = Dataset.from_list(load_jsonl('eval_data.jsonl'))
print(f"✅ Loaded {len(eval_dataset)} evaluation examples")


# STEP 2: Define perplexity function

def calculate_perplexity(model, tokenizer, dataset, max_samples=50):
    """Calculate perplexity on evaluation dataset"""

    model.eval()
    total_loss = 0
    total_tokens = 0

    # Use subset for faster evaluation
    num_samples = min(max_samples, len(dataset))
    test_samples = dataset.select(range(num_samples))

    print(f"Calculating perplexity on {num_samples} samples...")

    with torch.no_grad():
        for example in tqdm(test_samples, desc="Evaluating"):
            try:
                # Tokenize
                inputs = tokenizer(
                    example['text'],
                    return_tensors='pt',
                    truncation=True,
                    max_length=2048
                ).to(model.device)

                # Get loss
                outputs = model(**inputs, labels=inputs['input_ids'])
                loss = outputs.loss

                # Accumulate
                num_tokens = inputs['input_ids'].size(1)
                total_loss += loss.item() * num_tokens
                total_tokens += num_tokens

            except Exception as e:
                print(f"Skipping sample due to error: {e}")
                continue

    if total_tokens == 0:
        return float('inf')

    # Calculate perplexity
    avg_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(avg_loss))

    return perplexity.item()


# STEP 3: Run evaluation

print("\n" + "="*60)
print("PERPLEXITY EVALUATION")
print("="*60)

perplexity = calculate_perplexity(model, tokenizer, eval_dataset, max_samples=50)

print("\n" + "="*60)
print("RESULTS")
print("="*60)
print(f"\n📊 Fine-tuned Model Perplexity: {perplexity:.2f}")
print("\nInterpretation:")
if perplexity < 10:
    print("  ✅ Excellent - Very confident predictions")
elif perplexity < 15:
    print("  ✅ Good - Solid performance")
elif perplexity < 25:
    print("  ⚠️  Okay - Moderate confidence")
else:
    print("  ❌ Poor - Low confidence")

print("\n✅ Evaluation complete!")





Starting evaluation setup...
Looking for eval data in Google Drive...
✅ Copied eval_data.jsonl from Drive
✅ Loaded 1918 evaluation examples

PERPLEXITY EVALUATION
Calculating perplexity on 50 samples...


Evaluating: 100%|██████████| 50/50 [00:13<00:00,  3.65it/s]


RESULTS

📊 Fine-tuned Model Perplexity: 5.42

Interpretation:
  ✅ Excellent - Very confident predictions

✅ Evaluation complete!
